<a href="https://colab.research.google.com/github/Code-by-Mann/Naari_Suraksha_AI/blob/main/Naari_Suraksha_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 - Create folder structure
import os

os.makedirs("project/agents", exist_ok=True)
os.makedirs("project/tools", exist_ok=True)
os.makedirs("project/memory", exist_ok=True)
os.makedirs("project/core", exist_ok=True)

In [ ]:
# Cell 2 - project/__init__.py
%%writefile project/__init__.py
# Project package initializer


Writing project/__init__.py


In [ ]:
# Cell 3 - project/agents/__init__.py
%%writefile project/agents/__init__.py
# Agents package initializer


Writing project/agents/__init__.py


In [ ]:
# Cell 4 - project/tools/__init__.py
%%writefile project/tools/__init__.py
# Tools package initializer


Writing project/tools/__init__.py


In [ ]:
# Cell 5 - project/memory/__init__.py
%%writefile project/memory/__init__.py
# Memory package initializer


Writing project/memory/__init__.py


In [ ]:
# Cell 6 - project/core/__init__.py
%%writefile project/core/__init__.py
# Core package initializer


Writing project/core/__init__.py


In [ ]:
# Cell 7 - project/core/context_engineering.py
%%writefile project/core/context_engineering.py
from dataclasses import dataclass, field
from typing import Any, Dict, Optional


@dataclass
class Context:
    """Lightweight context object passed between agents."""

    user_id: str
    session_id: str
    user_input: str
    language: str = "en"
    metadata: Dict[str, Any] = field(default_factory=dict)


def build_base_context(
    user_id: str,
    session_id: str,
    user_input: str,
    language: str = "en",
    metadata: Optional[Dict[str, Any]] = None,
) -> Context:
    """Construct a baseline Context for the multi-agent pipeline."""

    if metadata is None:
        metadata = {}
    return Context(
        user_id=user_id,
        session_id=session_id,
        user_input=user_input,
        language=language,
        metadata=metadata,
    )


Writing project/core/context_engineering.py


In [ ]:
# Cell 8 - project/core/observability.py
%%writefile project/core/observability.py
import datetime
from typing import Any, Dict


def _timestamp() -> str:
    return datetime.datetime.utcnow().isoformat() + "Z"


def log_event(event_type: str, message: str, extra: Dict[str, Any] | None = None) -> None:
    """Simple console logger for demo purposes."""

    if extra is None:
        extra = {}
    print(f"[{_timestamp()}] [{event_type}] {message} | {extra}")


Writing project/core/observability.py


In [ ]:
# Cell 9 - project/core/a2a_protocol.py
%%writefile project/core/a2a_protocol.py
from dataclasses import dataclass, field
from typing import Any, Dict, Optional


@dataclass
class AgentMessage:
    """Simple Agent-to-Agent message envelope used by Planner, Worker, Evaluator."""

    sender: str
    receiver: str
    content: Dict[str, Any] = field(default_factory=dict)
    intent: Optional[str] = None
    message_id: Optional[str] = None
    trace_id: Optional[str] = None


Writing project/core/a2a_protocol.py


In [ ]:
# Cell 10 - project/memory/session_memory.py
%%writefile project/memory/session_memory.py
from typing import Any, Dict, Optional


class SessionMemory:
    """Extremely simple in-memory session + user store for the demo."""

    def __init__(self) -> None:
        self.sessions: Dict[str, Dict[str, Any]] = {}
        self.users: Dict[str, Dict[str, Any]] = {}

    def _session_key(self, user_id: str, session_id: str) -> str:
        return f"{user_id}:{session_id}"

    def get_session(self, user_id: str, session_id: str) -> Dict[str, Any]:
        key = self._session_key(user_id, session_id)
        if key not in self.sessions:
            self.sessions[key] = {
                "history": [],
                "state": {},
            }
        return self.sessions[key]

    def update_session(self, user_id: str, session_id: str, update: Dict[str, Any]) -> None:
        session = self.get_session(user_id, session_id)
        session["state"].update(update)

    def append_history(self, user_id: str, session_id: str, event: Dict[str, Any]) -> None:
        session = self.get_session(user_id, session_id)
        session["history"].append(event)

    def get_user_profile(self, user_id: str) -> Dict[str, Any]:
        if user_id not in self.users:
            self.users[user_id] = {
                "user_id": user_id,
                "preferred_language": "en",
                "medical_profile": {
                    "blood_group": "O+",
                    "conditions": [],
                },
                "govt_id_ref": "XXXX1234",
                "emergency_contacts": [],
            }
        return self.users[user_id]

    def update_user_profile(self, user_id: str, update: Dict[str, Any]) -> None:
        profile = self.get_user_profile(user_id)
        profile.update(update)

    def get_history(self, user_id: str, session_id: str) -> Optional[list]:
        return self.get_session(user_id, session_id).get("history", [])


Writing project/memory/session_memory.py


In [ ]:
# Cell 11 - project/tools/tools.py
%%writefile project/tools/tools.py
from typing import Any, Dict


def calculate_expression(expression: str) -> float:
    """Tiny calculator utility. In real systems, use a safe parser instead of eval."""

    try:
        return float(eval(expression, {"__builtins__": {}}))
    except Exception:
        return 0.0


def summarize(text: str, max_words: int = 12) -> str:
    """Naive summarizer that truncates to a limited number of words."""

    words = text.split()
    if len(words) <= max_words:
        return text
    return " ".join(words[:max_words]) + "..."


def detect_language(text: str) -> str:
    """Very naive language detector: always returns 'en' for this demo."""

    return "en"


def simple_nlu(text: str) -> Dict[str, Any]:
    """Very basic NLU to map phrases to intents."""

    lowered = text.lower()
    if any(trigger in lowered for trigger in ["help", "bachao", "save me", "panic"]):
        intent = "emergency"
    elif any(word in lowered for word in ["ride", "cab", "taxi"]):
        intent = "ride_request"
    elif any(word in lowered for word in ["report", "incident"]):
        intent = "incident_report"
    else:
        intent = "small_talk"

    return {"intent": intent, "raw_text": text}


def build_alert_text(user_text: str) -> str:
    """Simple placeholder for constructing an emergency alert-style message."""

    return (
        "🚨 Emergency detected. "
        "This is a demo alert constructed from your message: "
        f'"{user_text}"'
    )


def format_safe_ride_response() -> str:
    """Placeholder for safe ride response."""

    return (
        "🚖 Safe ride demo: I would match you with a verified female driver "
        "and share her details here in a real system."
    )


def format_incident_report_ack() -> str:
    """Placeholder for incident reporting acknowledgement."""

    return (
        "📝 Incident noted. In a full system, this would update the community "
        "safety map and your personal history."
    )


Writing project/tools/tools.py


In [ ]:
# Cell 12 - project/agents/planner.py
%%writefile project/agents/planner.py
from typing import Any, Dict

from project.core.context_engineering import Context
from project.core.a2a_protocol import AgentMessage
from project.core.observability import log_event
from project.memory.session_memory import SessionMemory
from project.tools.tools import simple_nlu, detect_language


class Planner:
    """Planner agent for Naari Suraksha.

    - Interprets the high-level scenario and selects an intent.
    - Prepares an AgentMessage for the Worker agent.
    """

    def __init__(self, memory: SessionMemory) -> None:
        self.memory = memory

    def plan(self, context: Context) -> AgentMessage:
        """Decide what to do with this user input by performing basic NLU."""

        detected_lang = detect_language(context.user_input)
        profile = self.memory.get_user_profile(context.user_id)

        nlu = simple_nlu(context.user_input)
        intent = nlu.get("intent", "small_talk")

        log_event(
            "planner.plan",
            f"Detected intent '{intent}' for user input.",
            extra={"user_id": context.user_id, "session_id": context.session_id},
        )

        self.memory.append_history(
            context.user_id,
            context.session_id,
            {
                "type": "planner_intent",
                "intent": intent,
                "user_input": context.user_input,
            },
        )

        content: Dict[str, Any] = {
            "nlu": nlu,
            "raw_input": context.user_input,
            "language": detected_lang or profile.get("preferred_language", "en"),
        }

        return AgentMessage(
            sender="planner",
            receiver="worker",
            content=content,
            intent=intent,
        )


Writing project/agents/planner.py


In [ ]:
# Cell 13 - project/agents/worker.py
%%writefile project/agents/worker.py
from typing import Any, Dict

from project.core.context_engineering import Context
from project.core.a2a_protocol import AgentMessage
from project.core.observability import log_event
from project.memory.session_memory import SessionMemory
from project.tools.tools import (
    summarize,
    build_alert_text,
    format_safe_ride_response,
    format_incident_report_ack,
)


class Worker:
    """Worker agent for Naari Suraksha.

    Executes the actual task determined by the Planner.
    For this demo, it's a simple rule-based handler.
    """

    def __init__(self, memory: SessionMemory) -> None:
        self.memory = memory

    def handle(self, message: AgentMessage, context: Context) -> AgentMessage:
        intent = message.intent or "small_talk"
        raw_input = message.content.get("raw_input", "")

        if intent == "emergency":
            response_text = build_alert_text(raw_input)
        elif intent == "ride_request":
            response_text = format_safe_ride_response()
        elif intent == "incident_report":
            response_text = format_incident_report_ack()
        else:
            summary = summarize(raw_input)
            response_text = (
                "👂 Naari Suraksha demo: I heard you say: " f'"{summary}"'
            )

        log_event(
            "worker.handle",
            f"Handled intent '{intent}'.",
            extra={"user_id": context.user_id, "session_id": context.session_id},
        )

        self.memory.append_history(
            context.user_id,
            context.session_id,
            {
                "type": "worker_response",
                "intent": intent,
                "response_text": response_text,
            },
        )

        content: Dict[str, Any] = {
            "response_text": response_text,
            "intent": intent,
        }

        return AgentMessage(
            sender="worker",
            receiver="evaluator",
            content=content,
            intent=intent,
        )


Writing project/agents/worker.py


In [ ]:
# Cell 14 - project/agents/evaluator.py
%%writefile project/agents/evaluator.py
from project.core.context_engineering import Context
from project.core.a2a_protocol import AgentMessage
from project.core.observability import log_event


class Evaluator:
    """Evaluator agent for Naari Suraksha.

    In a full system, this would apply safety & quality checks.
    Here, it simply logs and passes through the worker response.
    """

    def __init__(self) -> None:
        pass

    def evaluate(self, message: AgentMessage, context: Context) -> AgentMessage:
        intent = message.content.get("intent", "")
        response_text = message.content.get("response_text", "")

        log_event(
            "evaluator.evaluate",
            f"Evaluating response for intent '{intent}'.",
            extra={"user_id": context.user_id, "session_id": context.session_id},
        )

        final_text = response_text

        return AgentMessage(
            sender="evaluator",
            receiver="main_agent",
            content={"response_text": final_text, "intent": intent},
            intent=intent,
        )


Writing project/agents/evaluator.py


In [ ]:
# Cell 15 - project/main_agent.py
%%writefile project/main_agent.py
from typing import Any, Dict

from project.agents.planner import Planner
from project.agents.worker import Worker
from project.agents.evaluator import Evaluator
from project.memory.session_memory import SessionMemory
from project.core.context_engineering import build_base_context, Context
from project.core.observability import log_event


class MainAgent:
    """Main orchestrator for Naari Suraksha.

    - Builds the Context.
    - Runs Planner -> Worker -> Evaluator.
    - Returns a simple dict with final response.
    """

    def __init__(self) -> None:
        self.memory = SessionMemory()
        self.planner = Planner(self.memory)
        self.worker = Worker(self.memory)
        self.evaluator = Evaluator()

    def handle_message(self, user_input: str) -> Dict[str, Any]:
        user_id = "demo-user"
        session_id = "demo-session"

        context: Context = build_base_context(
            user_id=user_id,
            session_id=session_id,
            user_input=user_input,
        )

        log_event(
            "main_agent.handle_message",
            "Received new user input.",
            extra={"user_id": user_id, "session_id": session_id},
        )

        planner_message = self.planner.plan(context)
        worker_message = self.worker.handle(planner_message, context)
        final_message = self.evaluator.evaluate(worker_message, context)

        response_text = final_message.content.get("response_text", "")
        intent = final_message.content.get("intent", "")

        log_event(
            "main_agent.handle_message",
            "Returning final response.",
            extra={"intent": intent},
        )

        return {
            "response": response_text,
            "intent": intent,
        }


def run_agent(user_input: str):
    agent = MainAgent()
    result = agent.handle_message(user_input)
    return result["response"]


Writing project/main_agent.py


In [ ]:
# Cell 16 - project/app.py
%%writefile project/app.py
from project.main_agent import run_agent


def cli_demo() -> None:
    """Minimal CLI loop as a stand-in for a UI layer."""

    print("Naari Suraksha demo. Type a message, or press Enter to exit.")
    while True:
        try:
            text = input("You: ").strip()
        except EOFError:
            break
        if not text:
            break
        response = run_agent(text)
        print("Agent:", response)


if __name__ == "__main__":
    cli_demo()


Writing project/app.py


In [ ]:
# Cell 17 - project/run_demo.py
%%writefile project/run_demo.py
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

from project.main_agent import run_agent


if __name__ == "__main__":
    print(run_agent("Hello! This is a demo."))


Writing project/run_demo.py


In [ ]:
# Cell 18 - project/requirements.txt
%%writefile project/requirements.txt
# Minimal requirements for this Naari Suraksha demo project.
# Uses only the Python standard library for core logic.


Writing project/requirements.txt


In [ ]:
# Cell 19 - Test cell
from project.main_agent import run_agent

print(run_agent("Hello!"))


[2025-12-01T17:22:47.458877Z] [main_agent.handle_message] Received new user input. | {'user_id': 'demo-user', 'session_id': 'demo-session'}
[2025-12-01T17:22:47.459006Z] [planner.plan] Detected intent 'small_talk' for user input. | {'user_id': 'demo-user', 'session_id': 'demo-session'}
[2025-12-01T17:22:47.459048Z] [worker.handle] Handled intent 'small_talk'. | {'user_id': 'demo-user', 'session_id': 'demo-session'}
[2025-12-01T17:22:47.459068Z] [evaluator.evaluate] Evaluating response for intent 'small_talk'. | {'user_id': 'demo-user', 'session_id': 'demo-session'}
[2025-12-01T17:22:47.459080Z] [main_agent.handle_message] Returning final response. | {'intent': 'small_talk'}
👂 Naari Suraksha demo: I heard you say: "Hello!"


In [ ]:
# Cell 20 - Zip the project
!zip -r project.zip project


  adding: project/ (stored 0%)
  adding: project/tools/ (stored 0%)
  adding: project/tools/__init__.py (stored 0%)
  adding: project/tools/tools.py (deflated 56%)
  adding: project/tools/__pycache__/ (stored 0%)
  adding: project/tools/__pycache__/tools.cpython-312.pyc (deflated 44%)
  adding: project/tools/__pycache__/__init__.cpython-312.pyc (deflated 22%)
  adding: project/run_demo.py (deflated 32%)
  adding: project/app.py (deflated 42%)
  adding: project/core/ (stored 0%)
  adding: project/core/a2a_protocol.py (deflated 44%)
  adding: project/core/__init__.py (stored 0%)
  adding: project/core/observability.py (deflated 39%)
  adding: project/core/__pycache__/ (stored 0%)
  adding: project/core/__pycache__/a2a_protocol.cpython-312.pyc (deflated 30%)
  adding: project/core/__pycache__/__init__.cpython-312.pyc (deflated 23%)
  adding: project/core/__pycache__/observability.cpython-312.pyc (deflated 30%)
  adding: project/core/__pycache__/context_engineering.cpython-312.pyc (deflate